In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

from sklearn import svm

from sklearn.model_selection import RandomizedSearchCV

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes

properati = pd.read_csv('datos/caba_para_mapa.csv',error_bad_lines=False)
properati=properati.dropna().reset_index()

<IPython.core.display.Javascript object>

In [7]:
clf = svm.SVC()
scaler = StandardScaler()
scaler2= MinMaxScaler()
cant_buckets = 10

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)

# lo casteo a float porque si no tira error
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 
properati.groupby('categories_by_price').count()

,index,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,...,expenses,properati_url,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range
categories_by_price,,,,,,,,,,,,,,,,,,,,,
0.0,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
1.0,44,44,44,44,44,44,44,44,44,44,...,44,44,44,44,44,44,44,44,44,44
2.0,412,412,412,412,412,412,412,412,412,412,...,412,412,412,412,412,412,412,412,412,412
3.0,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,...,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030
4.0,595,595,595,595,595,595,595,595,595,595,...,595,595,595,595,595,595,595,595,595,595
5.0,240,240,240,240,240,240,240,240,240,240,...,240,240,240,240,240,240,240,240,240,240
6.0,78,78,78,78,78,78,78,78,78,78,...,78,78,78,78,78,78,78,78,78,78
7.0,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
8.0,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13


# Random Search

In [8]:
import numpy as np
from time import time
from scipy.stats import randint as sp_randint

#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['lat'],properati['lon'])
y = properati['categories_by_price']

X=scaler.fit_transform(X,y)
X=scaler2.fit_transform(X,y)
#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

True
True


In [9]:
C_range = np.arange(0.1,1000,0.1)
gamma_range = np.arange(10,20,1)

In [15]:
%%notify

from sklearn.model_selection import StratifiedShuffleSplit

param_grid ={"gamma": gamma_range, "C": C_range}

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

random_search = RandomizedSearchCV(clf,param_distributions=param_grid,n_iter=20,cv=5,n_jobs=-1)
start = time()
random_search.fit(X_train, y_train)

print("RandomizedSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(random_search.cv_results_['params'])))
score.report_single(random_search.cv_results_)

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


RandomizedSearchCV took 23.20 seconds for 20 candidate parameter settings.
Puesto: 1
Promedio training score: 0.856 (std: 0.007)
Promedio validation score: 0.579 (std: 0.009)
Promedio fit time: 1.166s
Hiper-parametros: {'C': 976.60000000000002, 'gamma': 15}

Puesto: 2
Promedio training score: 0.832 (std: 0.009)
Promedio validation score: 0.578 (std: 0.015)
Promedio fit time: 0.921s
Hiper-parametros: {'C': 609.90000000000009, 'gamma': 15}

Puesto: 3
Promedio training score: 0.826 (std: 0.007)
Promedio validation score: 0.575 (std: 0.018)
Promedio fit time: 0.759s
Hiper-parametros: {'C': 444.90000000000003, 'gamma': 16}

Puesto: 3
Promedio training score: 0.770 (std: 0.005)
Promedio validation score: 0.575 (std: 0.010)
Promedio fit time: 0.355s
Hiper-parametros: {'C': 122.3, 'gamma': 17}

Puesto: 5
Promedio training score: 0.818 (std: 0.008)
Promedio validation score: 0.574 (std: 0.010)
Promedio fit time: 0.867s
Hiper-parametros: {'C': 575.80000000000007, 'gamma': 14}

Puesto: 5
Promedio

<IPython.core.display.Javascript object>

In [16]:
mejor = random_search.best_estimator_

In [17]:
properati['svm']=random_search.predict(X,y) 

0.62195121951219512